In [1]:
from diviner import calib
from diviner import file_utils as fu

## first the melting itself

In [7]:
def rename_column(colname):
    dealwith = '_'.join(colname.split('_')[1:])
    if int(dealwith[0]) in range(1,7):
        return 'a'+dealwith
    else:
        return 'b'+dealwith

In [36]:
def merge_rdr_column(target_df, colstring, rdrr):
    coldata = rdrr.filter(regex='^'+colstring)
    coldata = coldata.rename(columns=rename_column)
    molten = pd.melt(coldata.reset_index(),
                     id_vars=['index'],
                     var_name='det',
                     value_name=colstring)
    return target_df.merge(molten, on=['index','det'])

In [38]:
import os
root = '/raid1/maye/coldregions/no_rad_correction/region1'
rdr_columns = ['clat','clon']
for rad_file in rad_files[:1]:
    tstr = fu.get_timestr(rad_file)
    rad = pd.read_hdf(rad_file, 'radiance')
    tb = pd.read_hdf(os.path.join(root, tstr+'_tb.h5'), 'tb')
    tindex = fu.fname_to_tindex(rad_file)
    rad = rad[calib.thermal_detectors][tindex]
    tb = tb[calib.thermal_detectors][tindex]
    molten_rad = pd.melt(rad.reset_index(), id_vars=['index'], var_name='det',
                         value_name='radiance')
    molten_tb = pd.melt(tb.reset_index(), id_vars=['index'], var_name='det',
                        value_name='tb')
    # molten_rad.sort(columns=['index'], inplace=True)
    # molten_tb.sort(columns=['index'], inplace=True)
    radtb = molten_tb.merge(molten_rad, on=['index','det'])
    
    # get old rdr data
    rdrr = fu.RDRR_Reader(tstr).open()
    for col in rdr_columns:
        radtb = merge_rdr_column(radtb, col, rdrr)


In [39]:
jdate = rdrr.filter(regex='jdate')

In [41]:
jdate.sort_index().head(

,jdate
2009-07-08 08:00:00.053000,2455020.833334
2009-07-08 08:00:00.181000,2455020.833335
2009-07-08 08:00:00.309000,2455020.833337
2009-07-08 08:00:00.437000,2455020.833338
2009-07-08 08:00:00.563000,2455020.833340


In [42]:
radtb.head()

,index,det,tb,radiance,clat,clon
0,2009-07-08 08:00:00.053000,a3_01,230.474970,0.155516,-64.00552,73.15205
1,2009-07-08 08:00:00.181000,a3_01,236.284917,0.189423,-64.01242,73.15222
2,2009-07-08 08:00:00.309000,a3_01,236.135538,0.188481,-64.01933,73.15240
3,2009-07-08 08:00:00.437000,a3_01,224.855220,0.127260,-64.02624,73.15259
4,2009-07-08 08:00:00.563000,a3_01,206.155471,0.060387,-64.03309,73.15277


In [45]:
radtb = radtb.merge(jdate.reset_index(), on=['index'])

In [46]:
radtb.head()

,index,det,tb,radiance,clat,clon,jdate
0,2009-07-08 08:00:00.053000,a3_01,230.474970,0.155516,-64.00552,73.15205,2455020.833334
1,2009-07-08 08:00:00.053000,a3_02,193.719911,0.033977,-64.00562,73.13978,2455020.833334
2,2009-07-08 08:00:00.053000,a3_03,194.339885,0.035027,-64.00571,73.12753,2455020.833334
3,2009-07-08 08:00:00.053000,a3_04,221.081573,0.110597,-64.00581,73.11439,2455020.833334
4,2009-07-08 08:00:00.053000,a3_05,223.849916,0.122647,-64.00592,73.10085,2455020.833334


In [49]:
reload(calib)

<module 'diviner.calib' from '/u/paige/maye/src/diviner/calib.py'>

In [54]:
radtb['c'] = radtb.det.str[:2].map(calib.mcs_div_mapping)

In [56]:
radtb.tail()

,index,det,tb,radiance,clat,clon,jdate,c
2165431,2009-07-08 08:59:59.843000,a6_17,86.678675,0.001401,68.74474,252.86575,2455020.874998,6
2165432,2009-07-08 08:59:59.843000,a6_18,86.625016,0.001394,68.74448,252.92398,2455020.874998,6
2165433,2009-07-08 08:59:59.843000,a6_19,88.195254,0.001615,68.74421,252.98221,2455020.874998,6
2165434,2009-07-08 08:59:59.843000,a6_20,85.001098,0.001189,68.74395,253.03462,2455020.874998,6
2165435,2009-07-08 08:59:59.843000,a6_21,84.009621,0.001077,68.74368,253.08560,2455020.874998,6


In [60]:
radtb['det'] = radtb.det.str[-2:].astype('int')

In [61]:
radtb.head()

,index,det,tb,radiance,clat,clon,jdate,c
0,2009-07-08 08:00:00.053000,1,230.474970,0.155516,-64.00552,73.15205,2455020.833334,3
1,2009-07-08 08:00:00.053000,2,193.719911,0.033977,-64.00562,73.13978,2455020.833334,3
2,2009-07-08 08:00:00.053000,3,194.339885,0.035027,-64.00571,73.12753,2455020.833334,3
3,2009-07-08 08:00:00.053000,4,221.081573,0.110597,-64.00581,73.11439,2455020.833334,3
4,2009-07-08 08:00:00.053000,5,223.849916,0.122647,-64.00592,73.10085,2455020.833334,3


In [63]:
radtb.to_csv?

In [65]:
radtb.rename(columns={'index':'isotime'})

,isotime,det,tb,radiance,clat,clon,jdate,c
0,2009-07-08 08:00:00.053000,1,230.474970,0.155516,-64.00552,73.15205,2455020.833334,3
1,2009-07-08 08:00:00.053000,2,193.719911,0.033977,-64.00562,73.13978,2455020.833334,3
2,2009-07-08 08:00:00.053000,3,194.339885,0.035027,-64.00571,73.12753,2455020.833334,3
3,2009-07-08 08:00:00.053000,4,221.081573,0.110597,-64.00581,73.11439,2455020.833334,3
4,2009-07-08 08:00:00.053000,5,223.849916,0.122647,-64.00592,73.10085,2455020.833334,3
5,2009-07-08 08:00:00.053000,6,236.029305,0.187811,-64.00603,73.08730,2455020.833334,3
6,2009-07-08 08:00:00.053000,7,237.110412,0.194646,-64.00613,73.07377,2455020.833334,3
7,2009-07-08 08:00:00.053000,8,242.335474,0.230302,-64.00623,73.06064,2455020.833334,3
8,2009-07-08 08:00:00.053000,9,235.331605,0.183514,-64.00632,73.04839,2455020.833334,3
9,2009-07-08 08:00:00.053000,10,233.913044,0.174964,-64.00642,73.03526,2455020.833334,3


In [2]:
radfile = '/raid1/maye/coldregions/no_rad_correction/region1/2011020301_radiance.h5'

In [3]:
tstr = fu.get_timestr(radfile)
tstr

'2011020301'

In [68]:
reload(fu)

<module 'diviner.file_utils' from '/u/paige/maye/src/diviner/file_utils.py'>

In [69]:
rdrr = fu.RDRR_Reader(tstr).open()

In [70]:
rdrr.index

<class 'pandas.tseries.index.DatetimeIndex'>
[2011-02-03 01:00:00.080000, ..., 2011-02-03 01:59:59.744000]
Length: 28123, Freq: None, Timezone: None

In [5]:
import os
from diviner import file_utils as fu

In [7]:
rad_file = '/raid1/maye/coldregions/no_rad_correction/region3/2011061009_radiance.h5'
path = os.path.dirname(rad_file)
tstr = fu.get_timestr(rad_file)

In [8]:
rdrr = fu.RDRR_Reader(tstr).open()

In [39]:
for i in range(60):
    if len(rdrr[rdrr.index.second == i]) > 8:
        print i

32


In [44]:
len(rdrr.ix['2011-06-10 09:00:32.216000'])

27873

In [53]:
from datetime import timedelta

In [54]:
timedelta(minutes=59)

datetime.timedelta(0, 3540)

In [56]:
(rdrr.index[-1] - rdrr.index[0]) > timedelta(seconds=40)

False